In [ ]:
# En caso de no tenerlo
# % pip install mlflow

In [1]:
import mlflow
import mlflow.pyfunc
import pandas as pd
import joblib

In [ ]:
test = pd.read_csv('/home/project/data/processed/test/test-dengue.csv')
scaler = joblib.load('/home/project/deploy/scaler.pkl')
df = test.drop('TIPO_PACIENTE', axis = 1)
data = pd.get_dummies(df, columns=['SEXO', 'ESTATUS_CASO', 'RESULTADO_PCR'], drop_first=True).sample(5)
data = scaler.transform(data)

Configurar MLFlow para acceder al servidor de Dagshub que contiene los modelos

In [16]:
mlflow.set_tracking_uri("https://dagshub.com/empleo-inducido/project-ml.mlflow")

Dentro de variables `model_name` y `model_version` se guardan como estan etiquedos en MLFLow.

In [18]:
! export MODEL_NAME="testing-model"
! export MODEL_VERSION=1

# Construir la variable MODEL_URI
! export MODEL_URI="models:/${MODEL_NAME}/${MODEL_VERSION}"
! export RUN_ID="bagging"

# Ejecutar el comando mlflow models serve
! mlflow models serve -m 'runs:/cb5a75c8402747eb8e952c8bdce93e23/model'

Traceback (most recent call last):
  File "/usr/local/bin/mlflow", line 8, in <module>
    sys.exit(cli())
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/usr/local/lib/python3.10/site-packages/mlflow/models/cli.py", line 103, in serve
    return get_flavo